<a href="https://colab.research.google.com/github/sdbrgo/Glossi/blob/main/glossi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Glossi, My Hair Care Buddy**




1. Import Necessary Libraries

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

2. Import Dataset

In [ ]:
data = "" #insert name of the dataset
df = pd.read_csv(data)
df.head()

3. Preprocess Data

*   **Encode** nominal values.